In [1]:

import pickle
import pandas as pd
import numpy as np


import torch as tc
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [2]:

with open('data/data_res.pickle', 'rb') as pk:
    data = pickle.load(pk)

In [3]:
from collections import defaultdict
data_reorg = defaultdict(list)

for user in data['x']:
    for i,game in enumerate(data['x'][user]):
        if i+1 <= len(data['x'][user]) - 1:
            data_reorg['x_t'].append( data['x'][user][i] )
            data_reorg['y_t'].append( data['y'][user][i] )
            data_reorg['u_t'].append( data['u'][user][i] )
            data_reorg['x_t1'].append( data['x'][user][i+1] )
            data_reorg['y_t1'].append( data['y'][user][i+1] )
            
df = pd.DataFrame( data_reorg )


In [4]:

TRAINING_SAMPLE_SIZE = int(0.75*len(df))
train_idx = np.random.randint(0, len(df)-1, TRAINING_SAMPLE_SIZE)
verify_idx = np.setdiff1d(np.arange(len(df)), train_idx)

df.head()

,x_t,y_t,u_t,x_t1,y_t1
0,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [42]:

class Net(tc.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = tc.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = tc.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

train_data = df.iloc[train_idx]
verify_data = df.iloc[verify_idx]
x_dim = 1
y_dim = len(train_data['y_t'].iloc[0])
u_dim = len(train_data['u_t'].iloc[0])

h_input_layer_dim = x_dim+u_dim
nn_hidden_width = 100
h_output_layer_dim = y_dim


In [38]:
x_set, u_set = np.asarray(train_data['x_t'].to_list()), np.asarray(train_data['u_t'].to_list())
y_set = np.asarray(train_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_data = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_data = tc.tensor(y_set, dtype=tc.float)

In [47]:

net = nn.Sequential(nn.Linear(h_input_layer_dim, nn_hidden_width), 
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, h_output_layer_dim))

optimizer = tc.optim.Adam(net.parameters(), lr=0.005)
loss_func = tc.nn.MSELoss()  # this is for regression mean squared loss


for _ in range(10):
    optimizer.zero_grad()                 # clear gradients for next train
    v_pred = net(h_input_data)            # input x and predict based on x  
    loss = loss_func(v_pred, h_output_data)   # must be (1. nn output, 2. target)
    
    loss.backward()                            # backpropagation, compute gradients
    optimizer.step()                      # apply gradients
    print(f'eps: {_} -- {loss}')

eps: 0 -- 0.004399415105581284
eps: 1 -- 0.0021338798105716705
eps: 2 -- 0.0010622406844049692
eps: 3 -- 0.0008304956718347967
eps: 4 -- 0.0006046133348718286
eps: 5 -- 0.0004913890152238309
eps: 6 -- 0.0004761289746966213
eps: 7 -- 0.000467058940557763
eps: 8 -- 0.00045365377445705235
eps: 9 -- 0.00044069081195630133


In [48]:
x_set, u_set = np.asarray(verify_data['x_t'].to_list()), np.asarray(verify_data['u_t'].to_list())
y_set = np.asarray(verify_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_verify = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_verify = tc.tensor(y_set, dtype=tc.float)

In [49]:
v_pred = net(h_input_verify)
loss = loss_func(v_pred, h_output_verify)
print(loss)

tensor(0.0004, grad_fn=<MseLossBackward>)
